<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>

# Hyperparameter Tuning

## *Data Science Unit 4 Sprint 2 Assignment 4*

## Your Mission, should you choose to accept it...

To hyperparameter tune and extract every ounce of accuracy out of this telecom customer churn dataset: [Available Here](https://lambdaschool-data-science.s3.amazonaws.com/telco-churn/WA_Fn-UseC_-Telco-Customer-Churn+(1).csv)

## Requirements

- Load the data
- Clean the data if necessary (it will be)
- Create and fit a baseline Keras MLP model to the data.
- Hyperparameter tune (at least) the following parameters:
 - batch_size
 - training epochs
 - optimizer
 - learning rate (if applicable to optimizer)
 - momentum (if applicable to optimizer)
 - activation functions
 - network weight initialization
 - dropout regularization
 - number of neurons in the hidden layer
 
 You must use Grid Search and Cross Validation for your initial pass of the above hyperparameters
 
 Try and get the maximum accuracy possible out of this data! You'll save big telecoms millions! Doesn't that sound great?


In [1]:
import tensorflow as tf

In [2]:
import pandas as pd
import numpy as np


In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.callbacks import EarlyStopping


In [4]:
url = "https://lambdaschool-data-science.s3.amazonaws.com/telco-churn/WA_Fn-UseC_-Telco-Customer-Churn+(1).csv"

df = pd.read_csv(url)

In [5]:
df.isnull().any()

customerID          False
gender              False
SeniorCitizen       False
Partner             False
Dependents          False
tenure              False
PhoneService        False
MultipleLines       False
InternetService     False
OnlineSecurity      False
OnlineBackup        False
DeviceProtection    False
TechSupport         False
StreamingTV         False
StreamingMovies     False
Contract            False
PaperlessBilling    False
PaymentMethod       False
MonthlyCharges      False
TotalCharges        False
Churn               False
dtype: bool

In [6]:
df.describe(exclude='number')

,customerID,gender,Partner,Dependents,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,TotalCharges,Churn
count,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043
unique,7043,2,2,2,2,3,3,3,3,3,3,3,3,3,2,4,6531,2
top,0899-LIIBW,Male,No,No,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,,No
freq,1,3555,3641,4933,6361,3390,3096,3498,3088,3095,3473,2810,2785,3875,4171,2365,11,5174


In [7]:
df['MultipleLines'].unique()

array(['No phone service', 'No', 'Yes'], dtype=object)

In [8]:
df['OnlineSecurity'].unique()

array(['No', 'Yes', 'No internet service'], dtype=object)

In [9]:
df['TotalCharges'].value_counts()

           11
20.2       11
19.75       9
19.9        8
19.65       8
           ..
7767.25     1
8196.4      1
4209.95     1
24.6        1
2230.85     1
Name: TotalCharges, Length: 6531, dtype: int64

In [10]:
df['TotalCharges'] = df['TotalCharges'].replace(' ',0)

In [11]:
df['TotalCharges'] = df['TotalCharges'].astype('float')

In [12]:
df['MonthlyCharges'] = df['MonthlyCharges'].astype('float')

In [13]:
df = df.drop('customerID', axis=1)

In [14]:
df['Churn'] = df['Churn'].replace(["Yes", "No"], [1,0])

In [15]:
df.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,0
1,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.50,0
2,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,1
3,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,0
4,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,1


In [16]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder


train, test = train_test_split(df)

In [17]:
X_train = train.drop('Churn', axis=1)
y_train = train['Churn']
X_test = test.drop('Churn', axis=1)
y_test = test['Churn']

In [18]:
encoder = OneHotEncoder(handle_unknown='ignore')
X_train_encoded = encoder.fit_transform(X_train)
X_test_encoded = encoder.transform(X_test)
encoder.categories_

[array(['Female', 'Male'], dtype=object),
 array([0, 1], dtype=int64),
 array(['No', 'Yes'], dtype=object),
 array(['No', 'Yes'], dtype=object),
 array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
        34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
        51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
        68, 69, 70, 71, 72], dtype=int64),
 array(['No', 'Yes'], dtype=object),
 array(['No', 'No phone service', 'Yes'], dtype=object),
 array(['DSL', 'Fiber optic', 'No'], dtype=object),
 array(['No', 'No internet service', 'Yes'], dtype=object),
 array(['No', 'No internet service', 'Yes'], dtype=object),
 array(['No', 'No internet service', 'Yes'], dtype=object),
 array(['No', 'No internet service', 'Yes'], dtype=object),
 array(['No', 'No internet service', 'Yes'], dtype=object),
 array(['No', 'No internet service', 'Yes'], dtype=object),
 a

In [19]:
X_train_encoded = X_train_encoded.toarray()

In [20]:
X_train_encoded.shape

(5282, 6551)

In [21]:
X_test_encoded = X_test_encoded.toarray()

In [22]:
X_test_encoded.shape

(1761, 6551)

In [23]:
y_train.shape

(5282,)

In [39]:
model = Sequential()
model.add(Dense(1280, input_dim=X_train_encoded.shape[1], activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [40]:
stop = EarlyStopping(min_delta=0.01, patience=5)

In [41]:
model.fit(X_train_encoded, y_train, epochs=100, validation_data=(X_test_encoded, y_test), callbacks=[stop])

Train on 5282 samples, validate on 1761 samples
Epoch 1/100
5282/5282 [==============================] - 2s 335us/sample - loss: 0.4456 - accuracy: 0.7808 - val_loss: 0.4335 - val_accuracy: 0.7876
Epoch 2/100
5282/5282 [==============================] - 1s 268us/sample - loss: 0.3552 - accuracy: 0.8389 - val_loss: 0.4441 - val_accuracy: 0.7797
Epoch 3/100
5282/5282 [==============================] - 1s 272us/sample - loss: 0.2078 - accuracy: 0.9123 - val_loss: 0.5884 - val_accuracy: 0.7632
Epoch 4/100
5282/5282 [==============================] - 1s 281us/sample - loss: 0.0801 - accuracy: 0.9680 - val_loss: 0.7303 - val_accuracy: 0.7183
Epoch 5/100
5282/5282 [==============================] - 1s 274us/sample - loss: 0.0385 - accuracy: 0.9866 - val_loss: 1.0345 - val_accuracy: 0.7087
Epoch 6/100
5282/5282 [==============================] - 1s 279us/sample - loss: 0.0345 - accuracy: 0.9858 - val_loss: 0.9193 - val_accuracy: 0.7394


In [43]:
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

In [42]:
def create_model():
    model = Sequential()
    model.add(Dense(1280, input_dim=X_train_encoded.shape[1], activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [47]:
model = KerasClassifier(build_fn=create_model, verbose=0)

In [45]:


param_grid = {'batch_size': [10, 20, 40, 60, 80, 100],
              'epochs': [20]}

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X_train_encoded, y_train, validation_data=(X_test_encoded, y_test))

print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}") 

Train on 4225 samples, validate on 1761 samples
Epoch 1/20
4225/4225 [==============================] - 4s 834us/sample - loss: 0.4510 - accuracy: 0.7792 - val_loss: 0.4389 - val_accuracy: 0.7978
Epoch 2/20
4225/4225 [==============================] - 3s 778us/sample - loss: 0.3488 - accuracy: 0.8362 - val_loss: 0.4907 - val_accuracy: 0.7763
Epoch 3/20
4225/4225 [==============================] - 3s 772us/sample - loss: 0.2071 - accuracy: 0.9176 - val_loss: 0.6092 - val_accuracy: 0.7462
Epoch 4/20
4225/4225 [==============================] - 3s 776us/sample - loss: 0.0745 - accuracy: 0.9718 - val_loss: 0.9037 - val_accuracy: 0.7138
Epoch 5/20
4225/4225 [==============================] - 3s 772us/sample - loss: 0.0398 - accuracy: 0.9853 - val_loss: 0.9863 - val_accuracy: 0.6848
Epoch 6/20
4225/4225 [==============================] - 3s 782us/sample - loss: 0.0249 - accuracy: 0.9875 - val_loss: 1.1051 - val_accuracy: 0.7058
Epoch 7/20
4225/4225 [==============================] - 3s 745us

In [ ]:
param_grid = {'batch_size': [100],
              'epochs': [20, 40, 80, 100, 400]}

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X_train_encoded, y_train, validation_data=(X_test_encoded, y_test))

print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}") 

## Stretch Goals:

- Try to implement Random Search Hyperparameter Tuning on this dataset
- Try to implement Bayesian Optimiation tuning on this dataset using hyperas or hyperopt (if you're brave)
- Practice hyperparameter tuning other datasets that we have looked at. How high can you get MNIST? Above 99%?
- Study for the Sprint Challenge
 - Can you implement both perceptron and MLP models from scratch with forward and backpropagation?
 - Can you implement both perceptron and MLP models in keras and tune their hyperparameters with cross validation?